# Introduction/Business Problem

I would like to compare and determine the best neighborhood in San Antonio, TX for tourists. San Antonio is a big city with a number of tourist sites. It has a local NBA team San Antonio Spurs, which draws NBA fans from all over the world each year. Moreover, with the historical connection with Mexican culture, the Mexican food there is a must try. I hope to give tourists a good recommendation on where to stay given their specific interests in the city.

Whether the focus is on food, tourist sites, or watching NBA games, I would like to let the users to have the power to assign their priority and recommend neighborhoods accordingly. I believe this added flexibility will let this project help many people who visit San Antonio.





# Data

To do so, we need to first define the neighborhoods in San Antonio. The definition of it will come from www.sanantonio.com/neighborhoods, which gives 9 of it. I will first determine the coordinates of these 9 neighborhoods. Then, for each neighborhood, I shall explore the nearby venues. Given that people usually drive in San Antonio, we may be able to set a larger radius, say 2 miles for now. I am also contemplating searching through the city and give scores based on the distance.

Given 